# Import libraries

In [1]:
import pandas as pd
import altair as alt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
alt.renderers.enable("default")

RendererRegistry.enable('default')

# Import and clean data

## Import Data

In [2]:
crude_oil_data = pd.read_csv("U.S._crude_oil_production.csv")
crude_oil_data.sample(5)

,Month,U.S. Crude Oil,Alabama,Alaska,Arkansas,Arizona,California,Colorado,Federal Offshore Gulf of Mexico Crude Oil,Federal Offshore Pacific Crude Oil,...,Ohio,Oklahoma,Pennsylvania,South Dakota,Wyoming,West Virginia,Virginia,Utah,Texas,Tennessee
96,2016-06-01,8670,22,470,15,0,484,309,1551,17,...,63,433,16,4,193,21,0,80,3130,1
57,2013-03-01,7208,29,533,18,0,545,164,1255,50,...,16,320,13,5,174,15,0,90,2384,1
75,2014-09-01,9041,27,477,19,0,568,278,1422,51,...,44,400,19,5,222,34,0,116,3278,1
10,2009-04-01,5278,20,653,16,0,573,81,1465,63,...,13,188,8,5,143,4,0,64,1110,1
40,2011-10-01,5886,24,566,16,0,534,115,1291,55,...,13,224,9,5,152,6,0,75,1595,1


## Cleaning Data

In [3]:
crude_oil_data.isnull().sum()

Month                                        0
U.S. Crude Oil                               0
Alabama                                      0
Alaska                                       0
Arkansas                                     0
Arizona                                      0
California                                   0
Colorado                                     0
Federal Offshore Gulf of Mexico Crude Oil    0
Federal Offshore Pacific Crude Oil           0
Florida                                      0
Idaho                                        0
Illinois                                     0
Indiana                                      0
Kansas                                       0
Kentucky                                     0
Louisiana                                    0
Michigan                                     0
Mississippi                                  0
Missouri                                     0
Nebraska                                     0
Montana      

In [4]:
crude_oil_data.columns[crude_oil_data.sum() == 0]

Index(['Arizona', 'Virginia'], dtype='object')

In [5]:
crude_oil_data.drop(['Arizona', 'Virginia'], axis = 1, inplace=True)

In [6]:
crude_oil_data = crude_oil_data.rename(columns= {"U.S. Crude Oil ":"US_total", 
                                                 "Federal Offshore Gulf of Mexico Crude Oil": "Gulf_of_Mexico",
                                                "Federal Offshore Pacific Crude Oil":"Pacific_oil"})

crude_oil_data["Date"] = pd.to_datetime(crude_oil_data["Month"])
crude_oil_data.drop("Month", axis = 1, inplace = True)
crude_oil_data["Year"] = crude_oil_data["Date"].dt.year
crude_oil_data["Month"] = crude_oil_data["Date"].dt.month
crude_oil_data.sample(2)

,US_total,Alabama,Alaska,Arkansas,California,Colorado,Gulf_of_Mexico,Pacific_oil,Florida,Idaho,...,Pennsylvania,South Dakota,Wyoming,West Virginia,Utah,Texas,Tennessee,Date,Year,Month
74,8886,25,398,19,567,281,1435,54,6,0,...,19,5,215,29,114,3269,1,2014-08-01,2014,8
41,6001,25,593,17,538,118,1278,50,6,0,...,9,5,153,6,76,1658,1,2011-11-01,2011,11


In [7]:
crude_oil_data = crude_oil_data.melt(id_vars=["US_total","Date", "Year", "Month"],
                                     var_name="Area", value_name="Production")

In [8]:
crude_oil_data["Month_name"] = crude_oil_data["Date"].dt.month_name().str[:3]
crude_oil_data.sample(20)

,US_total,Date,Year,Month,Area,Production,Month_name
1070,8778,2016-12-01,2016,12,Idaho,1,Dec
1493,6001,2011-11-01,2011,11,Kentucky,3,Nov
2404,9140,2017-03-01,2017,3,Nevada,1,Mar
1504,6946,2012-10-01,2012,10,Kentucky,7,Oct
1561,9230,2017-07-01,2017,7,Kentucky,4,Jul
3318,6563,2012-09-01,2012,9,Wyoming,160,Sep
2700,5651,2011-08-01,2011,8,North Dakota,444,Aug
3599,9251,2015-12-01,2015,12,Utah,91,Dec
1042,8886,2014-08-01,2014,8,Idaho,0,Aug
786,7268,2013-06-01,2013,6,Pacific_oil,50,Jun


In [9]:
# crude_oil_data["Month"] = crude_oil_data["Month"].apply(str)
crude_oil_data.dtypes

US_total               int64
Date          datetime64[ns]
Year                   int64
Month                  int64
Area                  object
Production             int64
Month_name            object
dtype: object

In [10]:
columns_titles = ["Area","Production","US_total" ,"Year", "Month", "Date", "Month_name"]
crude_oil_data=crude_oil_data.reindex(columns=columns_titles)
crude_oil_data.reset_index()
crude_oil_data.head(2)

,Area,Production,US_total,Year,Month,Date,Month_name
0,Alabama,21,5138,2008,6,2008-06-01,Jun
1,Alabama,21,5177,2008,7,2008-07-01,Jul


In [11]:
sorted_production = crude_oil_data.groupby("Area",as_index=False).sum()\
.sort_values(by="Production", ascending = False).iloc[0:, 0:2].reset_index(drop=True)
#sorted_production

In [12]:
four_top = sorted_production["Area"][0:4]
four_top = list(four_top)
four_top

['Texas', 'Gulf_of_Mexico', 'North Dakota', 'Alaska']

In [13]:
line_data = crude_oil_data.\
groupby(["Year", "Area"], as_index=False).sum() # Date required for the average horizontal line

In [14]:
line_data["Ratio"] = (line_data["Production"] / line_data["US_total"] )* 100
line_data.groupby("Year").sum()

,Production,US_total,Month,Ratio
Year,,,,
2008,34212,1094752,2016,100.002923
2009,64170,2053760,2496,99.984419
2010,65735,2103712,2496,99.990873
2011,67826,2170528,2496,99.995577
2012,78009,2496224,2496,100.002564
2013,89591,2866720,2496,100.006698
2014,105068,3362048,2496,100.003807
2015,113173,3621728,2496,99.994699
2016,105975,3391040,2496,100.004718


# Interactive visualisations

## Define the filters

In [15]:
scale = alt.Scale(domain=['Alaska', 'North Dakota', 'Gulf_of_Mexico', 'Texas'],
                  range=['#999999', '#E69F00', '#CC79A7', '#56B4E9'])
slider = alt.binding_range(name= "Year", min = 2008, max = 2018, step = 1)
select_year = alt.selection_single(name = "Year", fields=["Year"],
                                  bind = slider, init = {"Year":2010})
month_name_sorted = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']

## Visuals

In [16]:
#brush = alt.selection(type = "interval", encodings = ["x"])
brush = alt.selection_multi(toggle="true",encodings=['x'])


bars = alt.Chart(data = crude_oil_data, height = 400, width=750)\
.mark_bar(color = "limegreen")\
.encode(x = alt.X("Area:O", sort="-y", axis=alt.Axis(title="Area")),
       y = alt.Y("sum(Production):Q",axis=alt.Axis(title="Total yearly production (BPD)")),
        tooltip= ["sum(Production)"],
       color = alt.condition(brush,
                              "sum(Production):Q",
                              alt.value("lightgray"),
                            scale = alt.Scale(scheme = "lighttealblue"),
                            legend = alt.Legend(title = "Total production", 
                                                orient="top", 
                                               titleAnchor='middle')))\
.add_selection(brush, select_year)\
.transform_filter(select_year)\

line = alt.Chart(data = line_data)\
.mark_rule(color = "#D81B60")\
.encode(y = "mean(Production):Q",
       size = alt.SizeValue(2),
       tooltip= ["mean(Production)"])\
.transform_filter(brush)\
.transform_filter(select_year)\
.properties(title= "Yearly Total Production by Area (BPD)")



mean_bars = alt.Chart(line_data)\
.mark_bar()\
.encode(alt.X(axis=alt.Axis(title="Area")),
        alt.Y('sum(Ratio):Q', axis=alt.Axis(title='Produciotn ratio to total over selected interval'),
              scale=alt.Scale(domain=(0, 100))),
        color=alt.value('#004D40'), tooltip = ["sum(Ratio):Q", "sum(Production)"])\
.transform_filter(brush)\
.transform_filter(select_year)\
.properties(width=25, height=400, title="Percentage of Total")

(bars + line) | mean_bars

alt.HConcatChart(...)

In [17]:
alt.Chart(data = crude_oil_data[crude_oil_data["Area"].isin(four_top)], height = 400, width = 800)\
.mark_bar()\
.encode(x = alt.X("Area:N", title = None, sort = "-y"),
       y = alt.Y("Production:Q", axis = alt.Axis(title = "Monthly production (BPD)")),
        column = alt.Column(field ="Month_name",sort = month_name_sorted ,title = None),
        color = alt.Color("Area:N", scale = scale,legend=alt.Legend(
        orient='none',
        legendX=0, legendY=-60,
        direction='horizontal',
        titleAnchor='middle')),
        tooltip = ["Production"])\
.properties(width = 65, title = "Monthy Crude Oil Production Data (BPD)")\
.configure_title(anchor = "middle")\
.add_selection(select_year)\
.transform_filter(select_year)\
.configure_facet(spacing =8)

alt.Chart(...)